# Proyecto

**Objetivos**:

* Realizar un análisis exploratorio de datos (EDA) con la finalidad de mejor la distribución de los datos utilizados (en este caso, las caracteristicas de los picos de los fringílidos a lo largo de los años y de la información hereditaria), como estan clasificados, su distribución, media, quartiles, etc.

* Determinar mediante una prueba de hipótesis si realmente hubo un cambio entre las caracteristicas del pico de estos animales a lo largo de los años.

* Adicional al punto anterior, tratar de determinar si existe una relación entre el las características físicas de los padres y aquellas de las crías.

In [1]:
# Librerias utiles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [2]:
df_1975 = pd.read_csv('./data/finch_beaks_1975.csv')
df_2012 = pd.read_csv('./data/finch_beaks_2012.csv')

df_fortis = pd.read_csv('./data/fortis_beak_depth_heredity.csv')
df_scandens = pd.read_csv('./data/scandens_beak_depth_heredity.csv')

In [3]:
df_1975.info() # No hay valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   band             403 non-null    int64  
 1   species          403 non-null    object 
 2   Beak length, mm  403 non-null    float64
 3   Beak depth, mm   403 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 12.7+ KB


In [4]:
df_2012.info() # Tampoco hay valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   band     248 non-null    int64  
 1   species  248 non-null    object 
 2   blength  248 non-null    float64
 3   bdepth   248 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 7.9+ KB


In [5]:
df_fortis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Mid-offspr  413 non-null    float64
 1   Male BD     413 non-null    float64
 2   Female BD   413 non-null    float64
dtypes: float64(3)
memory usage: 9.8 KB


In [6]:
df_scandens.info() # Tampoco

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mid_parent     130 non-null    float64
 1   mid_offspring  130 non-null    float64
dtypes: float64(2)
memory usage: 2.2 KB


## PARTE JOSE

## PARTE JESUS

## ¿Existe realmente una diferencia entre las aves de 1975 y las de 2012?

Uno de los objetivos presentados en este proyecto fue el de determinar si en realidad había una diferencia entre las caracteristicas de los picos de los fringílidos. Para esto podemos realizar una prueba de hipótesis, para tratar de determinar si la media tanto de la longitud como de la profundad de los picos en estas especies es diferente entre los años.

Sin embargo, como nuestro dataset incluye muestras de dos diferentes especies, es necesario separarlas de manera correspondiente.

In [7]:
# Separamos para la especie scandens
df_scandens_1975 = df_1975[df_1975['species'] == 'scandens']
df_scandens_2012 = df_2012[df_2012['species'] == 'scandens']

print(f"Tamaño de muestra G. scandens en 1975: {df_scandens_1975.shape[0]}")
print(f"Tamaño de muestra G. scandens en 2012: {df_scandens_2012.shape[0]}")

Tamaño de muestra G. scandens en 1975: 87
Tamaño de muestra G. scandens en 2012: 127


In [8]:
df_fortis_1975 = df_1975[df_1975['species'] == 'fortis']
df_fortis_2012 = df_2012[df_2012['species'] == 'fortis']

print(f"Tamaño de muestra G. fortis en 1975: {df_fortis_1975.shape[0]}")
print(f"Tamaño de muestra G. fortis en 2012: {df_fortis_2012.shape[0]}")

Tamaño de muestra G. fortis en 1975: 316
Tamaño de muestra G. fortis en 2012: 121


Una vez separadas nuestras muestras, podemos tratar de observar si existe alguna diferencia entre el promedio de éstas.

In [9]:
print(f"Longitud promedio del pico para G. fortis en 1975: {df_fortis_1975['Beak length, mm'].mean():.4f} mm")
print(f"Longitud promedio del pico para G. fortis en 2012: {df_fortis_2012['blength'].mean():.4f} mm")

Longitud promedio del pico para G. fortis en 1975: 10.5652 mm
Longitud promedio del pico para G. fortis en 2012: 10.5174 mm


In [10]:
print(f"Profundidad promedio del pico para G. fortis en 1975: {df_fortis_1975['Beak depth, mm'].mean():.4f} mm")
print(f"Profundidad promedio del pico para G. fortis en 2025: {df_fortis_2012['bdepth'].mean():.4f} mm")

Profundidad promedio del pico para G. fortis en 1975: 9.1716 mm
Profundidad promedio del pico para G. fortis en 2025: 8.6054 mm


A simple vista, puede parecer que tanto la longitud como la profundidad del pico de la especie *G. fortis* parecen haberse encogido, pero para determinar si hay una verdadera diferencia entre las aves en ambos años, podemos plantearnos la siguiente prueba de hipótesis.

$$ 
\begin{matrix}
H_0: & \mu_0 - \mu_1 = 0\\
H_1: & \mu_0 - \mu_1 > 0
\end{matrix}
$$

Donde $\mu_0$ y $\mu_1$ se refieren a la característica promedio de la especie en 1975 y en 2012, respectivamente. Aquí, la hipótesis nula indica que no existe una diferencia entre la característica promedio, es decir que $\mu_0 = \mu_1$, y la hipótesis alternativa propone que la característica del pico se ha encogido con el tiempo, o bien que $\mu_0 > \mu_1$, todo esto para un nivel de confiaza $\alpha = 0.05$.

En los casos donde comparamos dos medias poblacionales, el estadístico de prueba está dado por:

$$
z = \frac{\bar{X} - \bar{Y} - \Delta_0}{\sqrt{\frac{\sigma_{0}^{2}}{m} + \frac{\sigma_{1}^{2}}{n} }}
$$

En nuestro caso, los valores tanto de $\sigma_{0}^{2}$ como $\sigma_{1}^{2}$, los cuales representan las varianzas de ambas poblaciones, nos son desconocidos. Sin embargo, para tamaños de muestra suficientemente grandes, estos valores se pueden sustituir por las respectivas varianzas muestrales $S_{0}^{2}$ y $S_{1}^{2}$.

Por otro lado, $\Delta_0$ hace referencia a la diferencia de medias buscada, la cual en nuestro caso es igual a 0. Con estas nuevas consideraciones, podemos reescribir la expresión anterior como:

$$
z = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{S_{0}^{2}}{m} + \frac{S_{1}^{2}}{n} }}
$$

Asi, podemos comenzar a definir una funcion para obtener el valor de este estadistico:

In [11]:
def calcular_z(series1,series2):
    """
    Obtener el valor de z a partir de dos series de pandas
    """
    diff_prom = series1.mean() - series2.mean()
    
    valor1_raiz = (series1.var() / series1.size)
    valor2_raiz = (series2.var() / series2.size)
    
    z = diff_prom / np.sqrt(valor1_raiz + valor2_raiz)
    
    return z

Con esto podemos obtener facilmente el valor del estadístico de prueba tanto para la longitud como la profundidad del pico para la especie *G. fortis*:

In [18]:
z_long_fortis = calcular_z(df_fortis_1975['Beak length, mm'],df_fortis_2012['blength'])
z_prof_fortis = calcular_z(df_fortis_1975['Beak depth, mm'],df_fortis_2012['bdepth'])

print(f"Z para la longitud: {z_long_fortis:.3f}")
print(f"Z para la profundidad: {z_prof_fortis:.3f}")

Z para la longitud: 0.579
Z para la profundidad: 7.217


Una vez calculados los valores del estadístico de prueba, debemos obtener el valor crítico de z para saber si rechazaremos la hipótesis nula. Para el caso donde $H_1: \mu_0 - \mu_1 > 0$, el criterio de rechazo de la hipótesis nula para un valor determinado de $\alpha$ es cuando $z \geq z_{\alpha}$. La libreria ```scipy``` nos ofrece una función que nos ayuda a calcular este valor:

In [19]:
z_critico = stats.norm.ppf(0.95)
print(f"Z critico para 0.05: {z_critico:.3f}")

Z critico para 0.05: 1.645


Comparando este valor con los estadísticos de prueba para la longitud y para la profundidad de los picos, podemos hacer las siguientes afirmaciones para la especie *G. fortis*:

* Con los datos recabados, **existe eviencia para rechazar la hipótesis nula para el caso de la profundidad del pico**, con un nivel de confianza del 95%.

* Con los datos recabados, **no existe eviencia para rechazar la hipótesis nula para el caso de la longitud del pico**, con un nivel de confianza del 95%.

Estas afirmaciones nos dan a entender que existe evidencia de una disminución en los tamaños de profundidad del pico para las especies *G. fortis* que habitaban en 1975, y las que habitaban en el año 2012, mas no la existe para el caso de su longitud. Esto puede considerarse como evidencia de la evolución de estas especies a través del tiempo.

Así, podemos realizar el mismo procedimiento para el caso de la segunda especie, *G. scandens*, y sus respectivas longitudes y profundidades del pico:

In [15]:
print(f"Longitud promedio del pico para G. scandens en 1975: {df_scandens_1975['Beak length, mm'].mean():.4f} mm")
print(f"Longitud promedio del pico para G. scandens en 2012: {df_scandens_2012['blength'].mean():.4f} mm")

Longitud promedio del pico para G. scandens en 1975: 14.1209 mm
Longitud promedio del pico para G. scandens en 2012: 13.4210 mm


In [16]:
print(f"Profundidad promedio del pico para G. scandens en 1975: {df_scandens_1975['Beak depth, mm'].mean():.4f} mm")
print(f"Profundidad promedio del pico para G. scandens en 2025: {df_scandens_2012['bdepth'].mean():.4f} mm")

Profundidad promedio del pico para G. scandens en 1975: 8.9600 mm
Profundidad promedio del pico para G. scandens en 2025: 9.1862 mm


In [20]:
z_long_scandens = calcular_z(df_scandens_1975['Beak length, mm'],df_scandens_2012['blength'])
z_prof_scandens = calcular_z(df_scandens_1975['Beak depth, mm'],df_scandens_2012['bdepth'])

print(f"Z para la longitud: {z_long_scandens:.3f}")
print(f"Z para la profundidad: {z_prof_scandens:.3f}")

Z para la longitud: 6.787
Z para la profundidad: -2.663


Recordemos que el valor crítico para esta prueba es de 1.645, por lo que el criterio de rechazo $ z \geq z_{\alpha}$ se cumple para la longitud, mas no para la profundidad. Esto nos indica que, para *G. scandens*:

* Con los datos recabados, **existe eviencia para rechazar la hipótesis nula para el caso de la longitud del pico**, con un nivel de confianza del 95%.

* Con los datos recabados, **no existe eviencia para rechazar la hipótesis nula para el caso de la profundidad del pico**, con un nivel de confianza del 95%.

Con estas afirmaciones, podemos decir que existe evidencia de que la longitud del pico de la especie *G. scandens* se ha ido encogiendo a lo largo de los años, mas no es el caso de la profundidad del mismo.